### Извлечение сущностей с использованием Natasha и Yargy-parser

#### Готовые правила
Сейчас в Natasha есть правила для извлечения имён, адресов, дат, сумм денег, названий организаций и географических объектов.

#### Имена

In [ ]:
!pip install natasha==0.10.0 yargy==0.11.0

In [ ]:
from natasha import NamesExtractor
from natasha.markup import show_markup, show_json

extractor = NamesExtractor()

text = '''
Благодарственное письмо   
Хочу поблагодарить учителей моего, теперь уже бывшего, одиннадцатиклассника:  Бушуева Вячеслава Владимировича и Бушуеву Веру Константиновну. 
Они вовлекали сына в интересные внеурочные занятия, связанные с театром и походами.

Благодарю прекрасного учителя 1"А" класса - Волкову Наталью Николаевну, нашего наставника, тьютора - Ларису Ивановну!
За огромнейший труд, чуткое отношение к детям, взаимопонимание! Огромное спасибо!
'''

matches = extractor(text)
spans = [_.span for _ in matches]
print('Spans: ',spans)
facts = [_.fact.as_json for _ in matches]
print('Facts: ',facts)
show_markup(text, spans)
show_json(facts)


#### Адреса

In [ ]:
from natasha import AddressExtractor
from natasha.markup import show_markup, show_json

extractor = AddressExtractor()

text = '''
Предлагаю вернуть прежние границы природного парка №71 на Инженерной улице 2.

По адресу Алтуфьевское шоссе д.51 (основной вид разрешенного использования: производственная деятельность, склады) размещен МПЗ. Жители требуют незамедлительной остановки МПЗ и его вывода из района

Контакты О нас телефон 7 881 574-10-02 Адрес Республика Карелия,г.Петрозаводск,ул.Маршала Мерецкова, д.8 Б,офис 4
'''
matches = extractor(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
show_json(facts)

#### Правила для дат и денег

In [ ]:
from natasha import (
    NamesExtractor,
    AddressExtractor,
    DatesExtractor,
    MoneyExtractor
)
from natasha.markup import show_markup, show_json

extractors = [
    NamesExtractor(),
    AddressExtractor(),
    DatesExtractor(),
    MoneyExtractor()
]


text = '''
Взыскать к индивидуального предпринимателя Иванова Костантипа Петровича дата рождения 10 января 1970 года, проживающего по адресу город Санкт-Петербург, ул. Крузенштерна, дом 5/1А 8 000 (восемь тысяч) рублей 00 копеей госпошлины в пользу бюджета РФ 
'''
spans = []
facts = []
for extractor in extractors:
    matches = extractor(text)
    spans.extend(_.span for _ in matches)
    facts.extend(_.fact.as_json for _ in matches)
show_markup(text, spans)
show_json(facts)


#### Создание своих правил

#### Правило для извлечения дат в ISO-формате («2018-02-23», «2015-12-31»)

In [1]:
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte


DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)
YEAR = and_(
    gte(1),
    lte(2022)
)
DATE = rule(
    YEAR,
    '-',
    MONTH,
    '-',
    DAY
)

parser = Parser(DATE)
text = '''
2018-02-23,
2015-12-31;
8 916 364-12-01'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 11) ['2018', '-', '02', '-', '23']
[13, 23) ['2015', '-', '12', '-', '31']
[31, 40) ['364', '-', '12', '-', '01']


#### Корректировка правила

In [6]:
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte


DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)
YEAR = and_(
    gte(2000),
    lte(2022)
)
DATE = rule(
    YEAR,
    '-',
    MONTH,
    '-',
    DAY
)

parser = Parser(DATE)
text = '''
2018-02-23,
2015-12-31;
8 916 364-12-01'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 11) ['2018', '-', '02', '-', '23']
[13, 23) ['2015', '-', '12', '-', '31']


#### Правила для дат вида «8 января 2014», «15 июня 2001 г.»

In [11]:
from yargy import or_
from yargy.predicates import caseless, normalized, dictionary


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}
MONTH_NAME = dictionary(MONTHS)
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
)
DATE = or_(
    rule(
        YEAR,
        '-',
        MONTH,
        '-',
        DAY
    ),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS.optional()
    )
)

parser = Parser(DATE)
text = '''
8 января 2014 года, 15 июня 2001 г.,
31 февраля 2018'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 19) ['8', 'января', '2014', 'года']
[21, 36) ['15', 'июня', '2001', 'г', '.']
[38, 53) ['31', 'февраля', '2018']


#### Интерпретация

In [ ]:
from yargy.interpretation import fact


Date = fact(
    'Date',
    ['year', 'month', 'day', 'year_words']
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day
)
MONTH = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month
)
YEAR = and_(
    gte(1),
    lte(2018)
).interpretation(
    Date.year
)
MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month
)
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
).interpretation(
    Date.year_words
)
DATE = or_(
    rule(YEAR, '-', MONTH, '-', DAY),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS
    )
).interpretation(Date)

parser = Parser(DATE)
text = '''5 февраля 2011 года'''
for match in parser.findall(text):
    print(match.fact)


In [ ]:
match = parser.match('2018-12-01')
match.tree.as_dot